In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import regex as re
import string

In [ ]:
# Build SQL Connection
conn = sqlite3.connect('../data/GuidePod.sqlite')
cur = conn.cursor()

In [ ]:
# Get all table names from GuidePod4 file
tables_df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables_df

In [ ]:
# podcast_master
podcasts_df = pd.read_sql_query("SELECT * FROM podcast_master", conn)

# podcast_reviews
reviews_df = pd.read_sql_query("SELECT * FROM podcast_reviews", conn)

# episode_counts
episodes_df = pd.read_sql_query("SELECT * FROM episode_counts", conn)

# genre_master
genres_df = pd.read_sql_query("SELECT * FROM genre_master", conn)

# podcast_desc
descriptions_df = pd.read_sql_query("SELECT * FROM podcast_desc", conn)

# episode_details
episode_details_df = pd.read_sql_query("SELECT * FROM episode_details", conn)

## Podcast Data Cleanup

In [ ]:
podcasts_df

In [ ]:
# Check for duplicates from COUNT(*) and COUNT(unique podcast IDs)
cur.execute("""
SELECT
  COUNT(*)
, COUNT(DISTINCT id)
FROM podcast_master 
"""
)

print(cur.fetchall())

In [ ]:
# Creates a new dataframe to get clean data from podcasts_master (still duplicating 3 podcasts)

podcasts_clean = pd.read_sql_query("""
WITH rank as 
(
    SELECT 
      *
    , ROW_NUMBER() OVER (PARTITION BY name, id) as ranked
    FROM podcast_master 
    WHERE country = 'us'
    GROUP BY 1,2,3,4,5,6,7,8
)
SELECT
*
FROM rank
WHERE ranked = 1
""", conn)

podcasts_clean = podcasts_clean.drop(columns=['ranked'])
podcasts_clean.head()

> Use `podcasts_df` as the base dataset with all info in table

> Use `podcasts_clean` dataset for modeling/analyses

## Episode Counts Cleanup

In [ ]:
# Check out Row 417
episodes_df

In [ ]:
# Removed the row that contained header value
episodes_df = episodes_df[episodes_df.name != 'name']
episodes_df

In [ ]:
# One duplicate name, make sure to join on ID 
episodes_df.describe()

In [ ]:
# Confirming they are different episodes with different IDs
episodes_df[episodes_df.name == 'Motley Fool Money']

> `episodes_df` is clean to use for analysis

## Genre Master Cleanup

In [ ]:
genres_df

In [ ]:
# After Shows has two rows in here for whatever reason
genres_df.groupby('genre').size()

In [ ]:
# Removing duplicates and getting unique values only
genres_df = genres_df.drop_duplicates()
genres_df

> `genres_df` is clean to use for analysis

## Episode Details Cleanup

In [ ]:
# Need to clean out the duration so we get only the numbers instead of the string field

episode_details_df

In [ ]:
# Cleaning up the bad data
# Create new field that contains itunes or is null, and extracting the numerical duration from string

bad_durations = episode_details_df[episode_details_df['duration'].str.contains("itunes") | episode_details_df['duration'].isnull()]
bad_durations['new_duration'] = bad_durations['duration'].copy()
bad_durations['new_duration'] = bad_durations['new_duration'].str.extract('(\d+)')
bad_durations.head()

In [ ]:
# Updating episode_details_df with new_duration
episode_details_df['new_duration'] = episode_details_df['duration']
episode_details_df.update(bad_durations)

In [ ]:
# Some values in the bad_durations did not populate because they had no values
# Turning those values into numbers and imputing them with averages
# Taking one outlier and imputing with the average
# Averages were calculated in Excel = 20,200

avg_duration = 20200
episode_details_df.loc[episode_details_df['new_duration'].str.contains('duration', na=False), 'new_duration'] = avg_duration
episode_details_df.loc[episode_details_df['new_duration'] == 4294967295] = avg_duration
episode_details_df.loc[episode_details_df['new_duration'].isna()] = avg_duration
episode_details_df['new_duration'] = pd.to_numeric(episode_details_df['new_duration'])
episode_details_df.loc[episode_details_df['new_duration'] < 100] = avg_duration

In [ ]:
episode_details_df.dtypes

In [ ]:
episode_details_df.describe()

In [ ]:
episode_details_df.sort_values(by=['new_duration'])

> `episode_details_df` has a `new_duration` column that has the length of duration standardized in ints

In [ ]:
# Leaving the episode description, reviews, and reviews for Gary & Harjot's word parsing and 
# cleaning libraries to clean up.

## Reviews Cleanup

In [ ]:
reviews_df

In [ ]:
us_reviews = reviews_df[reviews_df['id'].isin(podcasts_clean['id']) & reviews_df['review_text'].notnull() & reviews_df['review_title'].notnull()]
us_reviews = us_reviews[['id','review_title','review_text']]
us_reviews['title'] = us_reviews.groupby(['id'], as_index=False)['review_title'].transform(lambda x : ' '.join(x))
us_reviews['text'] = us_reviews.groupby(['id'], as_index=False)['review_text'].transform(lambda x : ' '.join(x))

In [ ]:
us_reviews = us_reviews[['id','title','text']].drop_duplicates()
reviews_clean = us_reviews.merge(descriptions_df, left_on = 'id', right_on='podcast_id', how='left')
reviews_clean = reviews_clean[['id','title','text','description']]

In [ ]:
cols = ['title', 'text', 'description']
reviews_clean['combined'] = reviews_clean[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
reviews_clean = reviews_clean[['id','combined']]

In [ ]:
reviews_score = reviews_df[['id','review_rating']].groupby('id').agg(['count','mean']).reset_index()
reviews_score.columns = ['id','num_reviews','avg_review_score']
reviews_score

In [ ]:
reviews_final = reviews_clean.merge(reviews_score, on = 'id', how = 'left')
reviews_final

In [ ]:
def onlyWords(s):
    return re.sub(r'[^\w\s]', ' ', s).strip().lower()

In [ ]:
reviews_final.loc[:, 'combined'] = reviews_final.loc[:, 'combined'].apply(lambda x: onlyWords(x))

In [ ]:
reviews_final

## Episode Count Cleanup

In [ ]:
# Get episode counts 
episode_counts = episode_details_df[['podcast_id','title']].groupby('podcast_id').count().reset_index()
episode_counts.columns = ['podcast_id','episode_count']
episode_counts

In [ ]:
podcast_duration = episode_details_df[['podcast_id','new_duration']].groupby('podcast_id').sum().reset_index()
podcast_duration.columns = ['podcast_id','total_duration']
podcast_duration

In [ ]:
episodes_final = episode_counts.merge(podcast_duration, on = 'podcast_id')
episodes_final

## Final Cleanup

In [ ]:
# Use podcasts_df as main
# JOIN episodes_df for number of episodes
# JOIN descriptions_df for description


podcasts_cols = podcasts_clean[['id','name','releaseDate','Primary_Genre','Artist']]
#episodes_cols = episodes_df[['id','episode_count']]
#final_df = podcasts_cols.merge(episodes_cols, on = 'id', how = 'left')
final_df = podcasts_cols.merge(episodes_final, left_on = 'id', right_on = 'podcast_id', how = 'inner')
final_df = final_df.merge(reviews_final, on = 'id')
final_df.head()

In [ ]:
dummy = pd.get_dummies(final_df['Primary_Genre'])
dummy.head()

In [ ]:
model_df = pd.concat([final_df,dummy], axis = 1)
model_df.drop('podcast_id', axis = 'columns', inplace=True)

In [ ]:
model_df.head()

In [ ]:
test = model_df.iloc[3,8]

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///..data/podcast_clean.sqlite', echo=True)
sqlite_connection = engine.connect()

In [ ]:
sqlite_table = "podcast_model_data"
model_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')